In [ ]:
import pandas as pd
import os
from tqdm import tqdm

print(os.getcwd())

In [ ]:
from openai import AzureOpenAI

In [ ]:
# import sentencepiece
# import tensorflow as tf
# import transformers 
# from transformers import TFBertModel

# from kobert_tokenizer import KoBERTTokenizer

In [ ]:
df = pd.read_json('./data/SFTlabel.json') 

# drop data_category column


df[['answer_count','contents']] = df['answer'].apply(pd.Series)
df = df.drop(columns=['data_category', 'question_count', 'data_id', 'question_type', 'answer'])
df.head()

In [ ]:
api_version = "2024-12-01-preview"

model_type_list = ['gpt-4o-mini', 'gpt-4o', 'o4-mini']
model_endpoint="https://seonwoo0808-9982-resource.cognitiveservices.azure.com/"
subscription_key = "EUJZQkTsVDdYX9eIgSK6l2JVJ1FzV6g70NSom0eKgRv8HV3f2wzvJQQJ99BFACHYHv6XJ3w3AAAAACOGkSfe"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=model_endpoint,
    api_key=subscription_key,
)


def gpt_ask(qustion, model_type):
    if model_type not in model_type_list:
        raise ValueError(f"Model type must be one of {model_type_list}")
    
    chat_prompt = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "You are an AI assistant that helps people find information."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": qustion
                }
            ]
        }
    ]
    response = client.chat.completions.create(
        messages=chat_prompt,
        max_tokens=800,
        temperature=1.0,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        model=model_type
    )

    return response.choices[0].message.content
    

In [ ]:
# gpt_ask(df["question"][0], "gpt-4o")

In [ ]:
# copy df and add a new column for answers
result = df.copy()
result['gpt_answer'] = None
for i in tqdm(range(len(df))):
    result.loc[i, 'gpt_answer'] = gpt_ask(df["question"][i], "gpt-4o")

In [ ]:
result[0:19]

In [ ]:
# save the result to a csv file
result.to_csv('./data/SFTlabel_gpt4o.csv', index=False)